# Merging processed data
This notebook relies on the data from the previous notebook (but there is no need to run the previous notebook for this one to work however).

In [11]:
import gnssvod as gv
import pandas as pd
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates

## Merge
In the previous notebook, we processed raw RINEX observation files individually for each receiver and saved the results in corresponding NetCDF files.

In the case of a GNSS-VOD set up, receivers are analysed as pairs. One receiver lies above the forest canopy and provides a clear-sky reference, and the other one lies below the canopy and measures the forest attenuation.

Here we merge the data from these two receivers before making any plots. We also save the merged data in chunks that are always the same (for example we save them in daily chunks). This makes it easier to manipulate data and avoids relying on the temporal chunks with which data was initially logged (here data was logged in hourly log files that span from xx:07 too xx+1:06).

### gv.gather_stations()
This function will do several things
- It will read processed observation files that were saved in NetCDF format (output of "preprocess").
- It will combine data from the various receivers/stations according to user-specified pairing rules.
- It will only process data belonging to the requested time interval.
- It will return and/or save paired data in temporal chunks specified by the time interval.

#### Specifying input files

In [12]:
# first let's indicate where to find the data for each receiver
pattern={'MACROCOSM-5':'data_pr/nc/MACROCOSM-5*.nc',
        'MACROCOSM-1': 'data_pr/nc/MACROCOSM-1*.nc'}

#### Specifying time interval
Then we need to define the temporal interval and the temporal chunks we will want for the output data
                                                                             
Here we decide to process all data from '28-04-2021' to '29-04-2021', meaning 2 days, starting at '28-04-2021'

In [13]:
startday = start=pd.to_datetime('25-09-2023',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=2, freq='D', closed='left')
timeintervals

IntervalIndex([[2023-09-25 00:00:00, 2023-09-26 00:00:00), [2023-09-26 00:00:00, 2023-09-27 00:00:00)], dtype='interval[datetime64[ns], left]')

Using the timeintervals above will save/return the results in chunks of 1 day. If we wanted the results in hourly chunks, we could have written instead:

`timeintervals=pd.interval_range(start=startday, periods=48, freq='H', closed='left')`

Now the only thing left is to define how to combine the stations, using the same dictionary keys as in 'pattern'.

In [14]:
# define how to make pairs, always give reference station first, matching the dictionary keys of 'pattern'
pairings={'MACROCOSM':('MACROCOSM-5','MACROCOSM-1')}

# run function
out = gv.gather_stations(pattern,pairings,timeintervals)

Processing MACROCOSM
Listing the files matching with the interval
Found 1 files for MACROCOSM-5
Reading
Found 1 files for MACROCOSM-1
Reading
Concatenating


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:335: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out[case_name] = [x for x in iout.groupby(pd.cut(iout.index.get_level_values('Epoch').tolist(), timeintervals))]


The result is of the form

out = dict(key:list(
<br>&emsp;&emsp;tuple(pd.Interval,pd.DataFrame)),
<br>&emsp;&emsp;tuple(pd.Interval,pd.DataFrame)),
<br>&emsp;&emsp;tuple(pd.Interval,pd.DataFrame))
<br>)

In our case, something like:

out = dict('Dav': \[
<br>&emsp;&emsp;(Interval('2021-04-28', '2021-04-29', closed='left'), dataframe),
<br>&emsp;&emsp;(Interval('2021-04-29', '2021-04-30', closed='left'), dataframe)
<br>\])

In [16]:
out['MACROCOSM'][0][1]

S1C   S1X   S2C  S2X  Azimuth  Elevation
Station     Epoch               SV                                            
MACROCOSM-5 2023-09-25 13:26:15 C20   NaN   NaN   NaN  NaN   -107.4       33.5
                                C27   NaN   NaN   NaN  NaN    130.1       72.8
                                C30   NaN   NaN   NaN  NaN    -35.8       51.3
                                C32   NaN   NaN   NaN  NaN    -23.6       61.2
                                E04   NaN  45.8   NaN  NaN   -159.1       81.7
...                                   ...   ...   ...  ...      ...        ...
MACROCOSM-1 2023-09-25 14:24:00 R01  41.7   NaN  31.9  NaN    -58.5       48.7
                                R08  45.8   NaN  38.9  NaN     13.8       29.5
                                R22  29.8   NaN  25.4  NaN    146.0       33.3
                                R23  44.9   NaN   NaN  NaN      NaN        NaN
                                R24  47.9   NaN  38.8  NaN    -24.7       30.7

[10912 rows x 6 columns]

#### Specifying output destination
Instead of just returning the result as an output of the function, we can specify where to save it instead. Again it may also be useful to get rid of some variables that are not useful to reduce file size.

In [19]:
# define where to save output data, matching the dictionary keys in 'pairings'
outputdir = {'MACROCOSM':'data_pr/MACROCOSM_paired/'}
# define which variables to keep
keepvars = ['S1C','S2C','Azimuth','Elevation']

# run function
out = gv.gather_stations(pattern,pairings,timeintervals,keepvars=keepvars,outputdir=outputdir)

Processing MACROCOSM
Listing the files matching with the interval
Found 1 files for MACROCOSM-5
Reading
Found 1 files for MACROCOSM-1
Reading
Concatenating
Saving files for MACROCOSM in data_pr/MACROCOSM_paired/
Saved 10912 obs in MACROCOSM_20230925000000_20230926000000.nc
No data for timestep 20230926000000_20230927000000, no file saved


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:335: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out[case_name] = [x for x in iout.groupby(pd.cut(iout.index.get_level_values('Epoch').tolist(), timeintervals))]


In [20]:
out

{'MACROCOSM': [(Interval(2023-09-25 00:00:00, 2023-09-26 00:00:00, closed='left'),
                                         S1C   S2C  Azimuth  Elevation
   Station     Epoch               SV                                 
   MACROCOSM-5 2023-09-25 13:26:15 C20   NaN   NaN   -107.4       33.5
                                   C27   NaN   NaN    130.1       72.8
                                   C30   NaN   NaN    -35.8       51.3
                                   C32   NaN   NaN    -23.6       61.2
                                   E04   NaN   NaN   -159.1       81.7
   ...                                   ...   ...      ...        ...
   MACROCOSM-1 2023-09-25 14:24:00 R01  41.7  31.9    -58.5       48.7
                                   R08  45.8  38.9     13.8       29.5
                                   R22  29.8  25.4    146.0       33.3
                                   R23  44.9   NaN      NaN        NaN
                                   R24  47.9  38.8    -24.7      

As we asked, the results have been saved as daily files (even though the input files are hourly files). The file names are generated based on the key of the 'pairing' argument (here 'Dav') and the specified time intervals.